# The file is to select and integrate gym data.
## Instruction: The file generates two csv files - 'bs_train_attr.csv' and 'review_train.csv'. 'bs_train_attr.csv' is business_train data of gym with the attributes split from column 'attributes'. 'review_train.csv' is review_train data of gym.

In [1]:
#Import modules
import json
import re
import pandas as pd
import nltk
import math
from math import sqrt
import sklearn
import statsmodels.api as sm
from nltk.stem import WordNetLemmatizer 
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
import numpy as np
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn import linear_model
import collections
from gensim.models import Word2Vec
from gensim.models import word2vec
import multiprocessing
import time
import re
import csv
import numpy as np

### First, we read the 'business_train.json', and save it as 'bs_train'.Then we select the gym data, save it as 'business_train', and transform it as data frame.

In [2]:
#Read business_train.json file
bs_train = []
with open('../data/business_train.json',"r") as f:
        try:
            while True:
                line = f.readline()
                if line:
                    bs_tra = json.loads(line)
                    bs_train.append(bs_tra)
                    #print(bs_train)
                else:
                    break
        except:
            f.close()
bs_train = pd.DataFrame(bs_train)         #set bs_train data as data frame

In [3]:
#Select the gym data from the whole data set.
category = bs_train[["categories"]]           #Store the categories of bs_train in 'category'
pat_rest = re.compile(r'.*Gym.*')             #Match where there is 'Gym'
num_no_rest = []
for row in tqdm(range(len(category))):                  #Checking by row, if the 'categories' is not defined  
    if category.iloc[row]['categories'] is None:        #save it in row.   
        num_no_rest.append(row)
    else:
        if not pat_rest.match(category.iloc[row]['categories']):       #Checking by row, if the 'categories' is
            num_no_rest.append(row)                                    #not in category, save it in row.     
bs_train = bs_train.drop(index = num_no_rest)               #Drop the rows which are not clssified as 'Gym'.

100%|██████████| 154606/154606 [00:26<00:00, 5746.55it/s]


In [4]:
bs_train

,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,state
16,"{'AcceptsInsurance': 'True', 'ByAppointmentOnl...",104170,"Trainers, Health & Medical, Active Life, Physi...",Toronto,"{'Monday': '6:0-22:0', 'Tuesday': '6:0-22:0', ...",1,43.645041,-79.395799,Totum Life Science,M5V 1K4,ON
397,{'BusinessAcceptsCreditCards': 'True'},49708,"Active Life, Trainers, Fitness & Instruction, ...",North Las Vegas,"{'Monday': '15:30-19:30', 'Tuesday': '15:30-19...",1,36.276098,-115.203534,Iron Talon Crossfit,89084,NV
441,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...",173815,"Gyms, Fitness & Instruction, Active Life",Glendale,"{'Monday': '17:30-21:0', 'Tuesday': '17:30-21:...",1,33.520077,-112.184332,Relentless Power Gym,85301,AZ
494,"{'GoodForKids': 'False', 'ByAppointmentOnly': ...",183154,"Fitness & Instruction, Gyms, Active Life",Toronto,"{'Monday': '6:0-21:0', 'Tuesday': '6:0-21:0', ...",1,43.638122,-79.418156,F45 Training Liberty Village,M6K 3E7,ON
568,"{'BusinessParking': '{'garage': False, 'street...",138163,"Gyms, Active Life, Fitness & Instruction, Card...",Verdun,None,1,45.465383,-73.566886,Energie Cardio,H4G 1V1,QC
622,"{'GoodForKids': 'False', 'BusinessParking': '{...",17594,"Active Life, Cardio Classes, Fitness & Instruc...",Las Vegas,"{'Monday': '4:30-21:30', 'Tuesday': '4:30-20:3...",1,36.298839,-115.279247,Orange Theory Fitness Las Vegas - Centennial,89117,NV
681,"{'BikeParking': 'False', 'ByAppointmentOnly': ...",106407,"Taekwondo, Gyms, Martial Arts, Education, Acti...",Henderson,"{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'...",1,36.042930,-115.116226,WTM Taekwondo,89074,NV
710,"{'BusinessAcceptsCreditCards': 'True', 'GoodFo...",170000,"Sports Clubs, Local Services, Community Servic...",Charlotte,"{'Monday': '5:0-21:30', 'Tuesday': '5:0-21:30'...",1,35.315808,-80.778680,University City Branch YMCA,28262,NC
713,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...",69171,"Active Life, Gyms, Trainers, Fitness & Instruc...",Charlotte,"{'Monday': '6:0-21:0', 'Tuesday': '6:0-21:0', ...",1,35.101964,-80.991144,Fitness Together,28278,NC
801,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...",22591,"Gyms, Trainers, Active Life, Fitness & Instruc...",Henderson,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",1,36.035840,-115.098688,24 Hour Fitness - Wigwam Henderson,89074,NV


### Picking up the column 'attributes' of 'bs_train', we split it and take split attributes as separate columns. The corresponding data is Boolean value, saved as 'attr'.

In [10]:
#Have a look at the columns name of bs_train data.
bs_train.columns

Index(['attributes', 'business_id', 'categories', 'city', 'hours', 'is_open',
       'latitude', 'longitude', 'name', 'postal_code', 'state'],
      dtype='object')

In [11]:
#Split attributes by row, and save as new columns. 
bs_attr = list(dict(bs_train['attributes']).values())
attr = pd.DataFrame(bs_attr[0], index = [0])
for i in range(1,len(bs_attr)):
    attr = pd.concat([attr,pd.DataFrame(bs_attr[i],index = [i])])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


### Then, we find that the split attribute - 'BusinessParking' is also dictionary-structured and has not so many missing values. We split it as new columns with the values as boolen, saved as 'parking'.

In [13]:
#Pick up the BusinessParking column. Split it and take the results as new columns.
bs_parking = list(dict(attr['BusinessParking']).values())
parking = pd.DataFrame(np.nan, index = [0,1], columns = ['garage','street','validated','lot', 'valet'])
for i in range(2,len(bs_parking)):               #Check by row, if there is no value available, add a new row of 'nan'     
    if pd.isnull(bs_parking[i]) == True:
        parking.loc[i] = np.nan
    else:
        parking = pd.concat([parking,pd.DataFrame(eval(bs_parking[i]), index = [i])])   #If the value is available, split it 
        parking[i:(i+1)] = (parking[i:(i+1)] == 1.0)                                    # and add as new row.

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


### Next, we add the attributes from 'attr' and 'parking' to 'business_train'. We finally get the data with split attributes as seperate columns, saved as 'bs_train_attr'. Write it as 'bs_train_attr.csv'.

In [14]:
#Add the attributes as columns of business_train and save the new generated as business_train.
bs_train = bs_train.reset_index(drop=True)
bs_train_attr = pd.concat([bs_train,attr,parking],axis=1)
#Delet the dictionary-structured variables, which have been split as new columns.
bs_train_attr = bs_train_attr.drop(['BusinessParking','attributes'],axis=1) 
bs_train_attr.to_csv("bs_train_attr.csv",index = False)

### Last, we read review_train.json, and save it as 'rv_train'. Then we select the gum data from rv_train according to business id from 'bs_train'.

In [20]:
#Read review_train.json file
rv_train = []
with open('../data/review_train.json',"r") as f:
        try:
            while True:
                line = f.readline()
                if line:
                    rv_tra = json.loads(line)
                    rv_train.append(rv_tra)          
                else:
                    break
        except:
            f.close()
rv_train = pd.DataFrame(rv_train)          #set rv_train data as data frame.

In [21]:
#According to 'business_id' column of business_train, we selec the gym data from rv_train.
#and save it as review_train.
id_gym = []
id_review = rv_train['business_id']
id_business = list(bs_train['business_id'])       
for i in range(len(rv_train)):                    
    if id_review[i] in id_business:                 #Checking by row, judge if it is in the list of business_id.
        id_gym.append(i)                            #If so, store the index in id_gym.
review_train = rv_train.loc[id_gym]
review_train.to_csv("review_train.csv",index=False)